<a href="https://colab.research.google.com/github/austinkirwin/public-projects/blob/main/Python_projects/Titanic_project/Titanic_Survival.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Titanic Survival Analysis Project

In this project I will try and predict the survivors of the titanic test data set using linear regression. Additionally, I will identify which three variables explain survivability the most.

## Importing

In [2]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import statsmodels.formula.api as smf
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

## Reading in the training and test data

In [10]:
train_data = pd.read_csv("https://raw.githubusercontent.com/austinkirwin/public-projects/refs/heads/main/Python_projects/Titanic_project/train.csv")
test_data = pd.read_csv("http://raw.githubusercontent.com/austinkirwin/public-projects/refs/heads/main/Python_projects/Titanic_project/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## Linear Models

I'm going to use a linear model to predict which passengers survive and which do not.

In [26]:
# Splitting the training data and removing NaN values
train_data = train_data.dropna()
# Features matrix
train_feature = train_data.drop(['Survived'], axis = 1)
# Target variable
train_target = train_data['Survived']

In [27]:
# Dropping unnecessary variables
train_feature = train_feature.drop(['Name','Ticket','Cabin','Embarked','Sex'], axis = 1)

In [56]:
# Compiling and fitting the full model
full_model = LinearRegression()
full_model.fit(train_feature, train_target)

prediction_values = full_model.predict(test_data.drop(['Cabin','Embarked','Name','Sex','Ticket'], axis = 1).dropna(),)

In [71]:
final_preds = pd.DataFrame(prediction_values)
final_preds.columns = ['Survived']
final_preds

,Survived
0,1.0
1,1.0
2,0.0
3,1.0
4,1.0
...,...
326,1.0
327,1.0
328,1.0
329,1.0


In [73]:
# Reformatting each value to 'Yes' or 'No' for survival

final_preds[final_preds['Survived'] > .5] = 1
final_preds[final_preds['Survived'] < .5] = 0

map = {1: 'Yes', 0: 'No'}
final_preds['Survived'].map(map)

,Survived
0,Yes
1,Yes
2,No
3,Yes
4,Yes
...,...
326,Yes
327,Yes
328,Yes
329,Yes
